In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
ames = pd.read_csv("/content/AmesHousing.csv")
ames

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,...,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,NaN,8885,Pave,NaN,IR1,Low,...,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,132000
2928,2929,924100070,20,RL,77.0,10010,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,170000


In [ ]:
X = ames[["Gr Liv Area", "TotRms AbvGrd", "Bldg Type"]]
y = ames["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
from sklearn.compose import ColumnTransformer

#Model 1 - Size and # Rooms
ct = ColumnTransformer(
  [
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)


lr_pipeline_1 = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

lr_fitted = lr_pipeline_1.fit(X_train, y_train)
y_preds = lr_fitted.predict(X_test)
print(mean_squared_error(y_test, y_preds))

2505754042.33764


In [ ]:
#Model 2 - Size, # Rooms, Building Type
ct = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)


lr_pipeline_2 = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

lr_fitted = lr_pipeline_2.fit(X_train, y_train)
y_preds = lr_fitted.predict(X_test)
print(mean_squared_error(y_test, y_preds))

2359390802.557142


In [ ]:
#Model 3 - Size, Building Type, Interact
ct_dummies = ColumnTransformer(
  [("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])],
  remainder = "passthrough"
).set_output(transform = "pandas")


ct_inter = ColumnTransformer(
  [
    ("interaction", PolynomialFeatures(interaction_only = True), ["remainder__Gr Liv Area", "dummify__Bldg Type_1Fam"]),
  ],
  remainder = "drop"
).set_output(transform = "pandas")

lr_pipeline_3 = Pipeline(
  [("dummies", ct_dummies),
   ("interact", ct_inter),
   ("linear_regression", LinearRegression())]
)

lr_fitted = lr_pipeline_3.fit(X_train, y_train)
y_preds = lr_fitted.predict(X_test)
print(mean_squared_error(y_test, y_preds))

2530924535.931969


In [ ]:
#Model 4 - Degree 5 Size, Degree 5 # Rooms, Building Type
ct = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
).set_output(transform = "pandas")

ct_poly = ColumnTransformer(
    [
        ('degree5', PolynomialFeatures(5), ["standardize__Gr Liv Area", "standardize__TotRms AbvGrd"])
    ]
)

lr_pipeline_4 = Pipeline(
  [("preprocessing", ct),
   ("polynomials", ct_poly),
   ("linear_regression", LinearRegression())]
)

lr_fitted = lr_pipeline_4.fit(X_train, y_train)
y_preds = lr_fitted.predict(X_test)
print(mean_squared_error(y_test, y_preds))

2500555150.843146


It appears the second model, using size, building type, and number of rooms has the lowest RMSE and is therefore the best model.

# Part 2: 5 Fold Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score

X = ames.drop("SalePrice", axis = 1)
y = ames["SalePrice"]

scores = cross_val_score(lr_pipeline_1, X, y, cv=5, scoring='neg_mean_squared_error')
np.mean(np.sqrt(np.abs(scores)))

55806.32634926364

In [ ]:
scores = cross_val_score(lr_pipeline_2, X, y, cv=5, scoring='neg_mean_squared_error')
np.mean(np.sqrt(np.abs(scores)))

54140.66302092876

In [ ]:
scores = cross_val_score(lr_pipeline_3, X, y, cv=5, scoring='neg_mean_squared_error')
np.mean(np.sqrt(np.abs(scores)))

55807.63730068668

In [ ]:
scores = cross_val_score(lr_pipeline_4, X, y, cv=5, scoring='neg_mean_squared_error')
np.mean(np.sqrt(np.abs(scores)))

72227.37162995586

Taking the average RMSE of each of the five versions of each of the models, the one for model 2 is still the lowest. This method is better because it is more consistent.

# Part 3 Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

ct_poly = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("polynomial1", PolynomialFeatures(), ["Gr Liv Area"]),
    ("polynomial2", PolynomialFeatures(), ['TotRms AbvGrd'])
  ],
  remainder = "drop"
)

lr_pipeline_poly = Pipeline(
  [("preprocessing", ct_poly),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")

degrees = {'preprocessing__polynomial1__degree': np.arange(1, 10), 'preprocessing__polynomial2__degree': np.arange(1, 10)}
gscv = GridSearchCV(lr_pipeline_poly, degrees, cv = 5, scoring='r2')
gscv_fitted = gscv.fit(X, y)
print(gscv_fitted.cv_results_['mean_test_score'])

[ 0.53288244  0.53238285  0.53592417  0.54152875  0.54106618  0.53486226
  0.08006934 -1.09028712  0.27015513  0.53747194  0.53356735  0.53413413
  0.5354176   0.53026731  0.53331356  0.35249861 -0.17703818  0.49127631
  0.55764061  0.55685726  0.55403905  0.55039251  0.54654925  0.54517067
  0.44839944  0.42970276 -1.04472858  0.54952572  0.55034925  0.55062735
  0.55691006  0.55638804  0.55312822  0.55392528  0.27926936  0.03928049
  0.45186012  0.45186012  0.50520827  0.49662143  0.49269427  0.52243635
  0.51785306  0.43585891  0.4195445   0.33383743  0.33383744  0.33383744
  0.33383744  0.33383744  0.4864919   0.4918967   0.31200234  0.25841677
  0.02932179  0.02932175  0.02932173  0.02932181  0.02932183  0.02932168
  0.02932175  0.02932195  0.36287767 -0.96809676 -0.96809674 -0.968096
 -0.96809576 -0.96809542 -0.96809518 -0.96809571 -0.96809539 -0.96809646
 -4.54560402 -4.54560638 -4.54560434 -4.54561141 -4.54560405 -4.54560991
 -4.54560818 -4.54560676 -4.54560209]


In [ ]:
print(max(gscv_fitted.cv_results_['mean_test_score']))

0.5576406065380459


The best model is the one with degree 3 for size and degree 1 for number of rooms. The problem with this approach is it takes a long time to calculate. We should limit the values we try to lower degree polynomials that are less likely to be overfitted. We could also try increading the degree one at a time, then stop once it starts decreasing the accuracy